In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [2]:
# Read the csv file
weatherpy_csv = pd.read_csv("../WeatherPy/output_data/WeatherPy.csv")
# Convert to a dataframe
weatherpy_df = pd.DataFrame(weatherpy_csv)
# Display dataframe
weatherpy_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,fare,-16.70,-151.02,77.76,81,80,20.24,PF,1595199822
1,wainwright,52.84,-110.86,67.46,75,97,9.40,CA,1595199626
2,mahebourg,-20.41,57.70,68.00,77,75,14.99,MU,1595199376
3,kongoussi,13.33,-1.53,87.40,51,24,6.62,BF,1595199823
4,rawson,-43.30,-65.10,45.36,82,100,11.56,AR,1595199380
...,...,...,...,...,...,...,...,...,...
541,piet retief,-27.01,30.81,40.19,62,0,3.83,ZA,1595199857
542,moose factory,51.26,-80.61,64.40,93,90,9.17,CA,1595199857
543,bosansko grahovo,44.18,16.36,66.00,80,0,3.49,BA,1595199857
544,hrymayliv,49.33,26.01,66.00,69,95,2.91,UA,1595199857


In [3]:
# Store latitude and longitude values
location = weatherpy_df[["Lat", "Lng"]]
# Use humidity as weight
humidity = weatherpy_df["Humidity"]

In [4]:
# Plot Heatmap
humid_fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(location, weights=humidity, dissipating=False, max_intensity=100, point_radius=1.25)
# Add layer
humid_fig.add_layer(heat_layer)
# Display figure
humid_fig

Figure(layout=FigureLayout(height='420px'))

![title](images/humidity_heatmap.png)

In [5]:
# Narrow down dataframe to fit the folowing three weather conditions:
# (1) Wind speed less than 5 mph
# (2) Max temperature higher than 75 degrees but lower than 80
# (3) Humidity less than 80
ideal_df = weatherpy_df[(weatherpy_df["Wind Speed"] < 5.0) &
                        (weatherpy_df["Max Temp"] > 75.0) &
                        (weatherpy_df["Max Temp"] < 80.0) &
                        (weatherpy_df["Humidity"] < 80)
                         ]
# Drop any rows with missing data
ideal_df = ideal_df.dropna(how = "any")
# Reset index 
ideal_df = ideal_df.reset_index(drop = True)
# Display dataframe
ideal_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,saint-michel-des-saints,46.68,-73.92,78.01,8,83,1.01,CA,1595199829
1,pontes e lacerda,-15.23,-59.34,79.83,43,1,3.36,BR,1595199677
2,alta floresta,-9.88,-56.09,79.23,39,74,4.81,BR,1595199395
3,krasnyy yar,46.53,48.35,75.20,64,40,2.24,RU,1595199840
4,khandyga,62.67,135.60,78.85,68,5,3.06,RU,1595199717
5,jiaozuo,35.24,113.23,78.40,61,0,4.94,CN,1595199843
6,diapaga,12.07,1.79,76.64,77,14,4.36,BF,1595199846
7,aksarayskiy,46.79,48.01,75.20,64,40,2.24,RU,1595199847
8,corinto,37.94,22.96,77.00,58,0,1.01,GR,1595199848
9,dasoguz,41.84,59.97,77.00,44,9,4.70,TM,1595199852


In [6]:
# Create a copy of ideal_df to store into hotel_df
hotel_df = ideal_df.copy()
# Add new empty column "Hotel Name" to the dataframe
hotel_df["Hotel Name"] = ""
# Display dataframe
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,saint-michel-des-saints,46.68,-73.92,78.01,8,83,1.01,CA,1595199829,
1,pontes e lacerda,-15.23,-59.34,79.83,43,1,3.36,BR,1595199677,
2,alta floresta,-9.88,-56.09,79.23,39,74,4.81,BR,1595199395,
3,krasnyy yar,46.53,48.35,75.20,64,40,2.24,RU,1595199840,
4,khandyga,62.67,135.60,78.85,68,5,3.06,RU,1595199717,
5,jiaozuo,35.24,113.23,78.40,61,0,4.94,CN,1595199843,
6,diapaga,12.07,1.79,76.64,77,14,4.36,BF,1595199846,
7,aksarayskiy,46.79,48.01,75.20,64,40,2.24,RU,1595199847,
8,corinto,37.94,22.96,77.00,58,0,1.01,GR,1595199848,
9,dasoguz,41.84,59.97,77.00,44,9,4.70,TM,1595199852,


In [7]:
# Set url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Set parameters to search for Find the closest hotel using type "lodging" to coordinates
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Use iterrows to iterate through the hotel_df dataframe
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from dataframe 
    lat_hotel = row["Lat"]
    lng_hotel = row["Lng"]
    # Add coordinates of hotel to params dict
    params["location"] = f"{lat_hotel},{lng_hotel}"
    # Make API request
    print(f"Retrieving Results for Index {index}: {row['City'].title()}.")
    response = requests.get(base_url, params=params).json()
    # Extract results
    results = response['results']
    try:
        # Print out the closest hotel's name
        print(f"First hotel located within 5000 meters of {row['City'].title()} is {results[0]['name']}.")
        # Add hotel name to the dataframe
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    # Skip cities that return an error
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        # Drop the row from the dataframe
        hotel_df = hotel_df.drop([index], axis = 0)  
    print("------------")

Retrieving Results for Index 0: Saint-Michel-Des-Saints.
First hotel located within 5000 meters of Saint-Michel-Des-Saints is Motel Du Mont trinity.
------------
Retrieving Results for Index 1: Pontes E Lacerda.
First hotel located within 5000 meters of Pontes E Lacerda is Hotel Bressan.
------------
Retrieving Results for Index 2: Alta Floresta.
First hotel located within 5000 meters of Alta Floresta is Floresta Amazonica Hotel.
------------
Retrieving Results for Index 3: Krasnyy Yar.
First hotel located within 5000 meters of Krasnyy Yar is Cottege near Ufa.
------------
Retrieving Results for Index 4: Khandyga.
First hotel located within 5000 meters of Khandyga is Apel'sin.
------------
Retrieving Results for Index 5: Jiaozuo.
First hotel located within 5000 meters of Jiaozuo is Days Hotel and Suites Jiaozuo.
------------
Retrieving Results for Index 6: Diapaga.
Missing field/result... skipping.
------------
Retrieving Results for Index 7: Aksarayskiy.
Missing field/result... skippi

In [8]:
# Reset the index in case of dropped rows
hotel_df = hotel_df.reset_index(drop = True)
# Display dataframe
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,saint-michel-des-saints,46.68,-73.92,78.01,8,83,1.01,CA,1595199829,Motel Du Mont trinity
1,pontes e lacerda,-15.23,-59.34,79.83,43,1,3.36,BR,1595199677,Hotel Bressan
2,alta floresta,-9.88,-56.09,79.23,39,74,4.81,BR,1595199395,Floresta Amazonica Hotel
3,krasnyy yar,46.53,48.35,75.20,64,40,2.24,RU,1595199840,Cottege near Ufa
4,khandyga,62.67,135.60,78.85,68,5,3.06,RU,1595199717,Apel'sin
5,jiaozuo,35.24,113.23,78.40,61,0,4.94,CN,1595199843,Days Hotel and Suites Jiaozuo
6,corinto,37.94,22.96,77.00,58,0,1.01,GR,1595199848,Ephira Hotel
7,dasoguz,41.84,59.97,77.00,44,9,4.70,TM,1595199852,Hotel Erik
8,isangel,-19.55,169.27,75.20,73,40,4.70,VU,1595199375,Volcano Sea View Tree House


In [9]:
# Use the template to add hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the dataframe row with the template into variable 'hotel_info'
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Set
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add marker layer on top of heat map
humid_fig.add_layer(markers)
# Display figure
humid_fig

Figure(layout=FigureLayout(height='420px'))

![title](images/marker_heatmap.png)